In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# root_dir = "/content/drive/MyDrive/project"
# import os
# os.chdir(root_dir)

Mounted at /content/drive


In [ ]:
!pip install tensorflow==2.14.0
#!pip install librosa==0.9.2

ERROR: Could not find a version that satisfies the requirement tensorflow==2.14.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.19.0rc0)
ERROR: No matching distribution found for tensorflow==2.14.0


  Using cached librosa-0.9.2-py3-none-any.whl.metadata (8.2 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached resampy-0.4.3-py3-none-any.whl.metadata (3.0 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
Using cached librosa-0.9.2-py3-none-any.whl (214 kB)
Using cached audioread-3.0.1-py3-none-any.whl (23 kB)
Using cached pooch-1.8.2-py3-none-any.whl (64 kB)
Using cached resampy-0.4.3-py3-none-any.whl (3.1 MB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Heinzkoshy\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\Heinzkoshy\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Heinzkoshy\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Heinzkoshy\anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Heinzkoshy\anaconda3\Lib\http\client.py", line 479, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Users\Heinzkoshy\anaconda3\Lib\socket.py", line 720, in readinto
    return self._

In [9]:
!pip uninstall tensorflow


^C


In [8]:
import os
import numpy as np
import tensorflow as tf
from scipy.io import wavfile
from tqdm import tqdm
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, BatchNormalization, LeakyReLU, ReLU, Dropout, Concatenate, Multiply, Input
from tensorflow.keras.models import Model
from scipy.signal import resample

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:


def load_audio_data(dataset_path, target_sampling_rate=16000):
    x_list = []
    y_list = []

    for music_folder in tqdm(os.listdir(dataset_path), desc="Processing dataset"):
        music_path = os.path.join(dataset_path, music_folder)

        if os.path.isdir(music_path):

            mixture_path = os.path.join(music_path, "mixture.wav")
            drums_path = os.path.join(music_path, "bass.wav")

            if os.path.exists(mixture_path) and os.path.exists(drums_path):

                sr_mix, mixture = wavfile.read(mixture_path)
                sr_drums, drums = wavfile.read(drums_path)

                if sr_mix != target_sampling_rate:
                    mixture = resample(mixture, int(len(mixture) * target_sampling_rate / sr_mix))
                    sr_mix = target_sampling_rate

                if sr_drums != target_sampling_rate:
                    drums = resample(drums, int(len(drums) * target_sampling_rate / sr_drums))
                    sr_drums = target_sampling_rate

                # Convert to mono if stereo
                if len(mixture.shape) > 1:
                    mixture = mixture.mean(axis=1)
                if len(drums.shape) > 1:
                    drums = drums.mean(axis=1)

                mixture = mixture / np.max(np.abs(mixture))
                drums = drums / np.max(np.abs(drums))

                audio_chunks = [mixture[i:i+64] for i in range(0, len(mixture) - 64 + 1, 64)]
                drums_chunks = [drums[i:i+64] for i in range(0, len(drums) - 64 + 1, 64)]

                x_list.extend(audio_chunks)
                y_list.extend(drums_chunks)

                print(f"Processing '{music_folder}': {len(audio_chunks)} chunks")

    return np.array(x_list), np.array(y_list)


In [ ]:
train_dataset_path = "musdb18/train"
test_dataset_path = "musdb18/valid"

In [ ]:
x_train, y_train = load_audio_data(train_dataset_path)
x_test, y_test = load_audio_data(test_dataset_path)

Processing dataset:   1%|          | 1/100 [00:22<37:38, 22.81s/it]

Processing 'Young Griffo - Pennies': 69498 chunks


Processing dataset:   2%|▏         | 2/100 [00:39<31:09, 19.08s/it]

Processing 'Young Griffo - Blood To Bone': 63645 chunks


Processing dataset:   3%|▎         | 3/100 [00:51<25:42, 15.90s/it]

...

Processing 'The Districts - Vermont': 57038 chunks


Processing dataset:  94%|█████████▍| 15/16 [05:47<00:23, 23.89s/it]

Processing 'The Long Wait - Back Home To Blue': 65162 chunks


Processing dataset: 100%|██████████| 16/16 [06:01<00:00, 22.57s/it]

Processing 'The Wrong'Uns - Rothko': 50584 chunks


In [ ]:
# Save the datasets
np.save('x_train.npy', x_train)
np.save('y_train_bass.npy', y_train)
np.save('x_test.npy', x_test)
np.save('y_test_bass.npy', y_test)

print("Data saved successfully!")

Data saved successfully!


In [ ]:
x_train = np.load('x_train.npy')
y_train = np.load('y_train_bass.npy')
x_test = np.load('x_test.npy')
y_test = np.load('y_test_bass.npy')

print("Data loaded successfully!")


Data loaded successfully!


In [ ]:
x_train = np.expand_dims(x_train, -1)
y_train = np.expand_dims(y_train, -1)
x_test = np.expand_dims(x_test, -1)
y_test = np.expand_dims(y_test, -1)

In [ ]:
def build_model(input_shape):
    input_layer = Input(shape=input_shape)


    conv1 = Conv1D(filters=16, kernel_size=5, strides=1, padding='same')(input_layer)
    batch1 = BatchNormalization()(conv1)
    activ1 = LeakyReLU(alpha=0.2)(batch1)

    conv2 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(activ1)
    batch2 = BatchNormalization()(conv2)
    activ2 = LeakyReLU(alpha=0.2)(batch2)

    conv3 = Conv1D(filters=64, kernel_size=5, strides=1, padding='same')(activ2)
    batch3 = BatchNormalization()(conv3)
    activ3 = LeakyReLU(alpha=0.2)(batch3)

    conv4 = Conv1D(filters=128, kernel_size=5, strides=1, padding='same')(activ3)
    batch4 = BatchNormalization()(conv4)
    activ4 = LeakyReLU(alpha=0.2)(batch4)


    up3 = Conv1DTranspose(filters=64, kernel_size=5, strides=1, padding='same')(activ4)
    conc3 = Concatenate()([up3, activ3])
    uactiv3 = ReLU()(conc3)

    up2 = Conv1DTranspose(filters=32, kernel_size=5, strides=1, padding='same')(uactiv3)
    conc2 = Concatenate()([up2, activ2])
    uactiv2 = ReLU()(conc2)

    up1 = Conv1DTranspose(filters=16, kernel_size=5, strides=1, padding='same')(uactiv2)
    conc1 = Concatenate()([up1, activ1])
    uactiv1 = ReLU()(conc1)

    # Output
    mask = Conv1DTranspose(filters=1, kernel_size=5, strides=1, padding='same')(uactiv1)
    output_layer = Multiply()([input_layer, mask])

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Build and compile the model
model = build_model((64, 1))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss="mae")
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 64, 1)]              0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 64, 16)               96        ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 64, 16)               64        ['conv1d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 64, 16)               0         ['batch_normalization[0][0

In [ ]:
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_model_bass.h5', monitor='val_loss', save_best_only=True, verbose=1)
from tqdm.keras import TqdmCallback

history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=500,
    batch_size=32,
    verbose=0,
    callbacks=[lr_reduction, checkpoint, TqdmCallback()]
)


In [ ]:
model.save('unet_source_separation_bass.h5')
print("Model saved.")
